In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Fazer o download do dataset IMDB
# Este dataset está disponível nos Keras datasets: https://keras.io/api/datasets/imdb/
# A informação está completamente processada e pronta a usar pelo classificador

# O Dataset em bruto pode ser obtido aqui: https://ai.stanford.edu/~amaas/data/sentiment/
# Neste último caso seria necessário aplicar pré-processamento de texto antes de criar o classificador

tf.random.set_seed(42)

# Estes 2 parâmetros ajudam a definir a dimensão dos exemplos lidos 
max_features = 10000
common_words = 10

# Obter o dataset, dividindo-o 
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features, skip_top=common_words)

# Obter um dicionário que vai permitir descodificar as reviews
word_index = keras.datasets.imdb.get_word_index()

In [ ]:
# Verificar a dimensão dos conjuntos de treino e de teste

print('Train: ', x_train.shape)
print('Test: ', x_test.shape)

In [ ]:
# Visualizar algumas reviews. Altere o valor da variável review para aceder a outro texto

# O dataset já vem tratado e cada exemplo é um array numpy de inteiros, em que cada inteiro representa uma palavra
# A pontuação já foi removida, as palavras foram separadas por espaços e todos os caracteres foram convertidos para minúscula
# Os inteiros estão organizados por frequência, logo os menores inteiros correspondem a palavras mais frequentes
# Os inteiros 0, 1 e 2 tem um significado especial: padding, sos (starting of sequence), unk (unknown)

# O código mostra também a review descodificada, 
# uma vez que o método get_word_index() obtém um dicionário que mapeia as palavras para o seu valor numérico
# Labels: 0(Bad), 1(Good)

# Escolher a review
review = 4

print("Numero de palavras: " ,len(x_train[review]))
print(x_train[review])

tam = len(x_train[review])
print('Label ', y_train[review])


id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in x_train[review][:tam]])

In [ ]:
# Cortar as reviews para um determinado tamanho máximo
# Por omissão, o corte é feito no início e são mantidas apenas as maxlen palavras finais 

# O classificador terá que efetuar previsões tendo em consideração apenas as ultimas palavras da review
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

maxlen = 20

x_trainP = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_testP = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
# Visualizar algumas reviews
# O sentido positivo/negativo da review é evidente neste excerto de texto?

# Escolher a review
review = 10
tam = len(x_trainP[review])

print('Label ', y_train[review])
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in x_trainP[review][:tam]])


In [ ]:
# Criar uma rede baseline que processe os inputs de forma simples


keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

inputs = keras.Input(shape=[maxlen, 1])
x = keras.layers.Flatten()(inputs)

# Completar o modelo com as camadas escondidas

output = keras.layers.Dense(1, activation='sigmoid')(x)


model_1 = keras.Model(inputs, output)

In [ ]:
# Detalhes


model_1.summary()

In [ ]:
# Compilar e treinar o modelo 1

model_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model_1.fit(x_trainP, y_train, epochs=10, validation_split=0.2)

In [ ]:
# Avaliar o desempenho no conjunto de teste

model_1.evaluate(x_testP, y_test)

In [ ]:
# Adicionar uma camada para embedding. O treino do embedding será feito pelo classificador
# https://www.tensorflow.org/tutorials/text/word_embeddings
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

output_emb = 8

inputs = keras.Input(shape=[maxlen])
emb = keras.layers.Embedding(max_features,  output_emb)(inputs)
x = keras.layers.Flatten()(emb)

# Completar o modelo com as camadas escondidas

output = keras.layers.Dense(1, activation='sigmoid')(x)

model_2 = keras.Model(inputs, output)

In [ ]:
model_2.summary()

In [ ]:
model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model_2.fit(x_trainP, y_train,
                      epochs=10,
                      validation_split=0.2)


In [ ]:
# Avaliar o desempenho no conjunto de teste

model_2.evaluate(x_testP, y_test)

In [ ]:
# Substituir os nós da rede neuronal das camadas escondidas por células LSTM

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# A saída recebe as contribuições pesadas 

output_emb = 8

inputs = keras.Input(shape=[maxlen])
emb = keras.layers.Embedding(max_features,  output_emb)(inputs)

# Completar o modelo com as camadas escondidas

output = keras.layers.Dense(1, activation='sigmoid')(x)

model_3 = keras.Model(inputs, output)



In [ ]:
model_3.summary()

In [ ]:
model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model_3.fit(x_trainP, y_train,
                      epochs=10,
                      validation_split=0.2)

In [ ]:
# Avaliar o desempenho no conjunto de teste

model_3.evaluate(x_testP, y_test)

In [ ]:
# Utilizar um embedding pré-treinado
# Neste exemplo será usado o embedding GloVe: https://nlp.stanford.edu/projects/glove/

# Guardar o ficheiro com os embedding na diretoria corrente ('glove.6B.50d.txt') 
# O ficheiro está disponível no Moodle

embeddings_index = {}
f = open(os.path.join('glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# Criar uma matriz onde será colocado o embedding
# Tem max_words linhas e 50 colunas (a dimensão do embedding que vamos usar)

embedding_dim = 50
max_words = max_features
embedding_matrix = np.zeros((max_words, embedding_dim)) # matriz com zeros

# Preencher a matriz com os dados do embedding pré-treinado

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [ ]:
# Criar um modelo igual ao anterior, alterando apenas a dimensão do embedding

inputs = keras.Input(shape=[maxlen])
emb = keras.layers.Embedding(max_words, embedding_dim)(inputs)

# Completar o modelo com as camadas escondidas

output = keras.layers.Dense(1, activation='sigmoid')(x)

model_4 = keras.Model(inputs, output)


model_4.summary()

In [ ]:
# Colocar os valores do embedding pré-treinado na camada de embedding
# Congelar estes pesos para que não se alterem durante o treino

model_4.layers[1].set_weights([embedding_matrix])
model_4.layers[1].trainable = False

In [ ]:
model_4.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model_4.fit(x_trainP, y_train,
                      epochs=10,
                      validation_split=0.2)

In [ ]:
model_4.evaluate(x_testP, y_test)

In [ ]:
# Implementar e testar possíveis melhorias

# Variar alguns híper-parâmetros 

